In [1]:
import tensorflow as tf
import numpy as np
from tensorflow.contrib import rnn
import pprint 

pp = pprint.PrettyPrinter(indent=4)
sess = tf.InteractiveSession()

/anaconda3/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [ ]:
h = [1, 0, 0, 0]
e = [0, 1, 0, 0]
l = [0, 0, 1, 0]
o = [0, 0, 0, 1]

In [ ]:
with tf.variable_scope('one_cell') as scope:
    hidden_size = 2
    cell = tf.contrib.rnn.BasicRNNCell(num_units=hidden_size)
    
    x_data = np.array([[h]], dtype=np.float32)
    pp.pprint(x_data)
    outputs, _states = tf.nn.dynamic_rnn(cell, x_data, dtype=tf.float32)
    
    sess.run(tf.global_variables_initializer())
    pp.pprint(outputs.eval())

In [ ]:
with tf.variable_scope('n_sequences') as scope:
    hidden_size = 2
    cell = tf.contrib.rnn.BasicRNNCell(num_units=hidden_size)
    
    x_data = np.array([[h, e, l, l, o]], dtype=np.float32)
    print(x_data.shape)
    pp.pprint(x_data)
    outputs, _states = tf.nn.dynamic_rnn(cell, x_data, dtype=tf.float32)
    
    sess.run(tf.global_variables_initializer())
    pp.pprint(outputs.eval())

In [ ]:
with tf.variable_scope('batching_input') as scope:
    hidden_size = 2
    cell = tf.contrib.rnn.BasicRNNCell(num_units=hidden_size)
    
    x_data = np.array([[h, e, l, l, o],
                      [e, o, l, l, l],
                      [l, l, e, e, l]], dtype=np.float32)
    print(x_data.shape)
    pp.pprint(x_data)
    outputs, _states = tf.nn.dynamic_rnn(cell, x_data, dtype=tf.float32)
    
    sess.run(tf.global_variables_initializer())
    pp.pprint(outputs.eval())

In [ ]:
y_data = tf.constant([[1, 1, 1]])
prediction = tf.constant([[[0, 1], [1, 0], [0, 1]]], dtype=tf.float32)
weights = tf.constant([[1,1,1]], dtype=tf.float32)

sequence_loss = tf.contrib.seq2seq.sequence_loss(prediction, y_data, weights)
sess.run(tf.global_variables_initializer())
print("Loss: ", sequence_loss.eval())

In [ ]:
y_data = tf.constant([[1, 1, 1]])
prediction1 = tf.constant([[[0, 1], [0, 1], [0, 1]]], dtype=tf.float32)
prediction2 = tf.constant([[[1, 0], [1, 0], [1, 0]]], dtype=tf.float32)
prediction3 = tf.constant([[[0, 1], [1, 0], [0, 1]]], dtype=tf.float32)
weights = tf.constant([[1,1,1]], dtype=tf.float32)

sequence_loss1 = tf.contrib.seq2seq.sequence_loss(prediction1, y_data, weights)
sequence_loss2 = tf.contrib.seq2seq.sequence_loss(prediction2, y_data, weights)
sequence_loss3 = tf.contrib.seq2seq.sequence_loss(prediction3, y_data, weights)
sess.run(tf.global_variables_initializer())
print("Loss1: ", sequence_loss1.eval(),
     "Loss2: ", sequence_loss2.eval(),
     "Loss3: ", sequence_loss3.eval(),)

In [2]:
with tf.variable_scope('rnn_hihello') as scope:
    h = [1, 0, 0, 0, 0]
    i = [0, 1, 0, 0, 0]
    e = [0, 0, 1, 0, 0]
    l = [0, 0, 0, 1, 0]
    o = [0, 0, 0, 0, 1]
    hidden_size = 5
    input_dim = 5
    batch_size = 1
    sequence_length = 6
    idx2char = ['h', 'i', 'e', 'l', 'o']
    x_data = [[0, 1, 0, 2, 3, 3]]
    x_one_hot = [[h, i, h, e, l, l]]
    y_data = [[1, 0, 2, 3, 3, 4]]

    X = tf.placeholder(tf.float32, [None, sequence_length, input_dim])
    Y = tf.placeholder(tf.int32, [None, sequence_length])

    cell = tf.contrib.rnn.BasicLSTMCell(num_units=hidden_size, state_is_tuple=True)
    initial_state = cell.zero_state(batch_size, tf.float32)
    outputs, _states = tf.nn.dynamic_rnn(cell, X, initial_state=initial_state, dtype=tf.float32)

    weights = tf.ones([batch_size, sequence_length])

    sequence_loss = tf.contrib.seq2seq.sequence_loss(logits=outputs, targets=Y, weights=weights)

    loss = tf.reduce_mean(sequence_loss)

    train = tf.train.AdadeltaOptimizer(learning_rate=0.1).minimize(loss)

    prediction = tf.argmax(outputs, axis=2)
    
with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    for i in range(2000):
        l, _ = sess.run([loss, train], feed_dict={X: x_one_hot, Y: y_data})
        result = sess.run(prediction, feed_dict={X: x_one_hot})
        print(i, "loss:", l, "prediction: ", result, "true Y: ", y_data)

        result_str = [idx2char[c] for c in np.squeeze(result)]
        print("\tPrediction str: ", ''.join(result_str))

0 loss: 1.6098288 prediction:  [[3 0 3 3 1 1]] true Y:  [[1, 0, 2, 3, 3, 4]]
	Prediction str:  lhllii
1 loss: 1.6097913 prediction:  [[3 0 3 3 1 1]] true Y:  [[1, 0, 2, 3, 3, 4]]
	Prediction str:  lhllii
2 loss: 1.6097534 prediction:  [[3 0 3 3 1 1]] true Y:  [[1, 0, 2, 3, 3, 4]]
	Prediction str:  lhllii
3 loss: 1.6097152 prediction:  [[3 0 3 3 1 1]] true Y:  [[1, 0, 2, 3, 3, 4]]
	Prediction str:  lhllii
4 loss: 1.6096767 prediction:  [[3 0 3 3 1 1]] true Y:  [[1, 0, 2, 3, 3, 4]]
	Prediction str:  lhllii
5 loss: 1.6096381 prediction:  [[3 0 3 3 1 1]] true Y:  [[1, 0, 2, 3, 3, 4]]
	Prediction str:  lhllii
6 loss: 1.6095995 prediction:  [[3 0 3 3 1 1]] true Y:  [[1, 0, 2, 3, 3, 4]]
	Prediction str:  lhllii
7 loss: 1.6095604 prediction:  [[3 0 3 3 1 1]] true Y:  [[1, 0, 2, 3, 3, 4]]
	Prediction str:  lhllii
8 loss: 1.6095214 prediction:  [[3 0 3 3 1 1]] true Y:  [[1, 0, 2, 3, 3, 4]]
	Prediction str:  lhllii
9 loss: 1.6094822 prediction:  [[3 0 3 3 1 1]] true Y:  [[1, 0, 2, 3, 3, 4]]
	Pred

109 loss: 1.6053259 prediction:  [[3 0 3 3 1 2]] true Y:  [[1, 0, 2, 3, 3, 4]]
	Prediction str:  lhllie
110 loss: 1.6052825 prediction:  [[3 0 3 3 1 2]] true Y:  [[1, 0, 2, 3, 3, 4]]
	Prediction str:  lhllie
111 loss: 1.6052389 prediction:  [[3 0 3 3 1 2]] true Y:  [[1, 0, 2, 3, 3, 4]]
	Prediction str:  lhllie
112 loss: 1.6051954 prediction:  [[3 0 3 3 1 2]] true Y:  [[1, 0, 2, 3, 3, 4]]
	Prediction str:  lhllie
113 loss: 1.6051519 prediction:  [[3 0 3 3 1 2]] true Y:  [[1, 0, 2, 3, 3, 4]]
	Prediction str:  lhllie
114 loss: 1.6051083 prediction:  [[3 0 3 3 1 2]] true Y:  [[1, 0, 2, 3, 3, 4]]
	Prediction str:  lhllie
115 loss: 1.6050647 prediction:  [[3 0 3 3 1 2]] true Y:  [[1, 0, 2, 3, 3, 4]]
	Prediction str:  lhllie
116 loss: 1.6050209 prediction:  [[3 0 3 3 1 2]] true Y:  [[1, 0, 2, 3, 3, 4]]
	Prediction str:  lhllie
117 loss: 1.6049774 prediction:  [[3 0 3 3 1 2]] true Y:  [[1, 0, 2, 3, 3, 4]]
	Prediction str:  lhllie
118 loss: 1.6049335 prediction:  [[3 0 3 3 1 2]] true Y:  [[1, 0

191 loss: 1.601633 prediction:  [[3 0 3 3 1 2]] true Y:  [[1, 0, 2, 3, 3, 4]]
	Prediction str:  lhllie
192 loss: 1.6015861 prediction:  [[3 0 3 3 1 2]] true Y:  [[1, 0, 2, 3, 3, 4]]
	Prediction str:  lhllie
193 loss: 1.6015394 prediction:  [[3 0 3 3 1 2]] true Y:  [[1, 0, 2, 3, 3, 4]]
	Prediction str:  lhllie
194 loss: 1.6014926 prediction:  [[3 0 3 3 1 2]] true Y:  [[1, 0, 2, 3, 3, 4]]
	Prediction str:  lhllie
195 loss: 1.6014456 prediction:  [[3 0 3 3 1 2]] true Y:  [[1, 0, 2, 3, 3, 4]]
	Prediction str:  lhllie
196 loss: 1.6013987 prediction:  [[3 0 3 3 1 2]] true Y:  [[1, 0, 2, 3, 3, 4]]
	Prediction str:  lhllie
197 loss: 1.6013517 prediction:  [[3 0 3 3 1 2]] true Y:  [[1, 0, 2, 3, 3, 4]]
	Prediction str:  lhllie
198 loss: 1.6013048 prediction:  [[3 0 3 3 1 2]] true Y:  [[1, 0, 2, 3, 3, 4]]
	Prediction str:  lhllie
199 loss: 1.6012577 prediction:  [[3 0 3 3 1 2]] true Y:  [[1, 0, 2, 3, 3, 4]]
	Prediction str:  lhllie
200 loss: 1.6012106 prediction:  [[3 0 3 3 1 2]] true Y:  [[1, 0,

275 loss: 1.5975442 prediction:  [[3 0 3 3 2 2]] true Y:  [[1, 0, 2, 3, 3, 4]]
	Prediction str:  lhllee
276 loss: 1.5974935 prediction:  [[3 0 3 3 2 2]] true Y:  [[1, 0, 2, 3, 3, 4]]
	Prediction str:  lhllee
277 loss: 1.5974426 prediction:  [[3 0 3 3 2 2]] true Y:  [[1, 0, 2, 3, 3, 4]]
	Prediction str:  lhllee
278 loss: 1.5973918 prediction:  [[3 0 3 3 2 2]] true Y:  [[1, 0, 2, 3, 3, 4]]
	Prediction str:  lhllee
279 loss: 1.5973408 prediction:  [[3 0 3 3 2 2]] true Y:  [[1, 0, 2, 3, 3, 4]]
	Prediction str:  lhllee
280 loss: 1.5972898 prediction:  [[3 0 3 3 2 2]] true Y:  [[1, 0, 2, 3, 3, 4]]
	Prediction str:  lhllee
281 loss: 1.5972385 prediction:  [[3 0 3 3 2 2]] true Y:  [[1, 0, 2, 3, 3, 4]]
	Prediction str:  lhllee
282 loss: 1.5971875 prediction:  [[3 0 3 3 2 2]] true Y:  [[1, 0, 2, 3, 3, 4]]
	Prediction str:  lhllee
283 loss: 1.5971364 prediction:  [[3 0 3 3 2 2]] true Y:  [[1, 0, 2, 3, 3, 4]]
	Prediction str:  lhllee
284 loss: 1.5970852 prediction:  [[3 0 3 3 2 2]] true Y:  [[1, 0

356 loss: 1.5932416 prediction:  [[3 0 3 3 2 2]] true Y:  [[1, 0, 2, 3, 3, 4]]
	Prediction str:  lhllee
357 loss: 1.5931858 prediction:  [[3 0 3 3 2 2]] true Y:  [[1, 0, 2, 3, 3, 4]]
	Prediction str:  lhllee
358 loss: 1.5931301 prediction:  [[3 0 3 3 2 2]] true Y:  [[1, 0, 2, 3, 3, 4]]
	Prediction str:  lhllee
359 loss: 1.5930742 prediction:  [[3 0 3 3 2 2]] true Y:  [[1, 0, 2, 3, 3, 4]]
	Prediction str:  lhllee
360 loss: 1.5930183 prediction:  [[3 0 3 3 2 2]] true Y:  [[1, 0, 2, 3, 3, 4]]
	Prediction str:  lhllee
361 loss: 1.5929623 prediction:  [[3 0 3 3 2 2]] true Y:  [[1, 0, 2, 3, 3, 4]]
	Prediction str:  lhllee
362 loss: 1.5929062 prediction:  [[3 0 3 3 2 2]] true Y:  [[1, 0, 2, 3, 3, 4]]
	Prediction str:  lhllee
363 loss: 1.59285 prediction:  [[3 0 3 3 2 2]] true Y:  [[1, 0, 2, 3, 3, 4]]
	Prediction str:  lhllee
364 loss: 1.5927937 prediction:  [[3 0 3 3 2 2]] true Y:  [[1, 0, 2, 3, 3, 4]]
	Prediction str:  lhllee
365 loss: 1.5927372 prediction:  [[3 0 3 3 2 2]] true Y:  [[1, 0, 

...

1633 loss: 1.266892 prediction:  [[2 3 3 3 3 3]] true Y:  [[1, 0, 2, 3, 3, 4]]
	Prediction str:  elllll
1634 loss: 1.2665753 prediction:  [[2 3 3 3 3 3]] true Y:  [[1, 0, 2, 3, 3, 4]]
	Prediction str:  elllll
1635 loss: 1.2662591 prediction:  [[2 3 3 3 3 3]] true Y:  [[1, 0, 2, 3, 3, 4]]
	Prediction str:  elllll
1636 loss: 1.265943 prediction:  [[2 3 3 3 3 3]] true Y:  [[1, 0, 2, 3, 3, 4]]
	Prediction str:  elllll
1637 loss: 1.2656277 prediction:  [[2 3 3 3 3 3]] true Y:  [[1, 0, 2, 3, 3, 4]]
	Prediction str:  elllll
1638 loss: 1.2653127 prediction:  [[2 3 3 3 3 3]] true Y:  [[1, 0, 2, 3, 3, 4]]
	Prediction str:  elllll
1639 loss: 1.2649982 prediction:  [[2 3 3 3 3 3]] true Y:  [[1, 0, 2, 3, 3, 4]]
	Prediction str:  elllll
1640 loss: 1.2646837 prediction:  [[2 3 3 3 3 3]] true Y:  [[1, 0, 2, 3, 3, 4]]
	Prediction str:  elllll
1641 loss: 1.26437 prediction:  [[2 3 3 3 3 3]] true Y:  [[1, 0, 2, 3, 3, 4]]
	Prediction str:  elllll
1642 loss: 1.2640564 prediction:  [[2 3 3 3 3 3]] true Y:  

1713 loss: 1.2427315 prediction:  [[2 3 3 3 3 3]] true Y:  [[1, 0, 2, 3, 3, 4]]
	Prediction str:  elllll
1714 loss: 1.2424434 prediction:  [[2 3 3 3 3 3]] true Y:  [[1, 0, 2, 3, 3, 4]]
	Prediction str:  elllll
1715 loss: 1.2421557 prediction:  [[2 3 3 3 3 3]] true Y:  [[1, 0, 2, 3, 3, 4]]
	Prediction str:  elllll
1716 loss: 1.2418683 prediction:  [[2 3 3 3 3 3]] true Y:  [[1, 0, 2, 3, 3, 4]]
	Prediction str:  elllll
1717 loss: 1.2415811 prediction:  [[2 3 3 3 3 3]] true Y:  [[1, 0, 2, 3, 3, 4]]
	Prediction str:  elllll
1718 loss: 1.2412943 prediction:  [[2 3 3 3 3 3]] true Y:  [[1, 0, 2, 3, 3, 4]]
	Prediction str:  elllll
1719 loss: 1.2410077 prediction:  [[2 3 3 3 3 3]] true Y:  [[1, 0, 2, 3, 3, 4]]
	Prediction str:  elllll
1720 loss: 1.2407216 prediction:  [[2 3 3 3 3 3]] true Y:  [[1, 0, 2, 3, 3, 4]]
	Prediction str:  elllll
1721 loss: 1.2404357 prediction:  [[2 3 3 3 3 3]] true Y:  [[1, 0, 2, 3, 3, 4]]
	Prediction str:  elllll
1722 loss: 1.24015 prediction:  [[2 3 3 3 3 3]] true Y:

1792 loss: 1.2208723 prediction:  [[2 3 3 3 3 3]] true Y:  [[1, 0, 2, 3, 3, 4]]
	Prediction str:  elllll
1793 loss: 1.2206062 prediction:  [[2 3 3 3 3 3]] true Y:  [[1, 0, 2, 3, 3, 4]]
	Prediction str:  elllll
1794 loss: 1.2203405 prediction:  [[2 3 3 3 3 3]] true Y:  [[1, 0, 2, 3, 3, 4]]
	Prediction str:  elllll
1795 loss: 1.220075 prediction:  [[2 3 3 3 3 3]] true Y:  [[1, 0, 2, 3, 3, 4]]
	Prediction str:  elllll
1796 loss: 1.21981 prediction:  [[2 3 3 3 3 3]] true Y:  [[1, 0, 2, 3, 3, 4]]
	Prediction str:  elllll
1797 loss: 1.2195449 prediction:  [[2 3 3 3 3 3]] true Y:  [[1, 0, 2, 3, 3, 4]]
	Prediction str:  elllll
1798 loss: 1.2192801 prediction:  [[2 3 3 3 3 3]] true Y:  [[1, 0, 2, 3, 3, 4]]
	Prediction str:  elllll
1799 loss: 1.2190157 prediction:  [[2 3 3 3 3 3]] true Y:  [[1, 0, 2, 3, 3, 4]]
	Prediction str:  elllll
1800 loss: 1.2187514 prediction:  [[2 3 3 3 3 3]] true Y:  [[1, 0, 2, 3, 3, 4]]
	Prediction str:  elllll
1801 loss: 1.2184875 prediction:  [[2 3 3 3 3 3]] true Y: 

1871 loss: 1.2005754 prediction:  [[2 3 3 3 3 3]] true Y:  [[1, 0, 2, 3, 3, 4]]
	Prediction str:  elllll
1872 loss: 1.2003272 prediction:  [[2 3 3 3 3 3]] true Y:  [[1, 0, 2, 3, 3, 4]]
	Prediction str:  elllll
1873 loss: 1.2000792 prediction:  [[2 3 3 3 3 3]] true Y:  [[1, 0, 2, 3, 3, 4]]
	Prediction str:  elllll
1874 loss: 1.1998312 prediction:  [[2 3 3 3 3 3]] true Y:  [[1, 0, 2, 3, 3, 4]]
	Prediction str:  elllll
1875 loss: 1.1995839 prediction:  [[2 3 3 3 3 3]] true Y:  [[1, 0, 2, 3, 3, 4]]
	Prediction str:  elllll
1876 loss: 1.1993363 prediction:  [[2 3 3 3 3 3]] true Y:  [[1, 0, 2, 3, 3, 4]]
	Prediction str:  elllll
1877 loss: 1.1990893 prediction:  [[2 3 3 3 3 3]] true Y:  [[1, 0, 2, 3, 3, 4]]
	Prediction str:  elllll
1878 loss: 1.1988423 prediction:  [[2 3 3 3 3 3]] true Y:  [[1, 0, 2, 3, 3, 4]]
	Prediction str:  elllll
1879 loss: 1.1985955 prediction:  [[2 3 3 3 3 3]] true Y:  [[1, 0, 2, 3, 3, 4]]
	Prediction str:  elllll
1880 loss: 1.1983489 prediction:  [[2 3 3 3 3 3]] true 

1975 loss: 1.1758169 prediction:  [[2 3 3 3 3 3]] true Y:  [[1, 0, 2, 3, 3, 4]]
	Prediction str:  elllll
1976 loss: 1.1755887 prediction:  [[2 3 3 3 3 3]] true Y:  [[1, 0, 2, 3, 3, 4]]
	Prediction str:  elllll
1977 loss: 1.1753609 prediction:  [[2 3 3 3 3 3]] true Y:  [[1, 0, 2, 3, 3, 4]]
	Prediction str:  elllll
1978 loss: 1.1751332 prediction:  [[2 3 3 3 3 3]] true Y:  [[1, 0, 2, 3, 3, 4]]
	Prediction str:  elllll
1979 loss: 1.1749057 prediction:  [[2 3 3 3 3 3]] true Y:  [[1, 0, 2, 3, 3, 4]]
	Prediction str:  elllll
1980 loss: 1.1746783 prediction:  [[2 3 3 3 3 3]] true Y:  [[1, 0, 2, 3, 3, 4]]
	Prediction str:  elllll
1981 loss: 1.1744512 prediction:  [[2 3 3 3 3 3]] true Y:  [[1, 0, 2, 3, 3, 4]]
	Prediction str:  elllll
1982 loss: 1.1742244 prediction:  [[2 3 3 3 3 3]] true Y:  [[1, 0, 2, 3, 3, 4]]
	Prediction str:  elllll
1983 loss: 1.1739975 prediction:  [[2 3 3 3 3 3]] true Y:  [[1, 0, 2, 3, 3, 4]]
	Prediction str:  elllll
1984 loss: 1.1737709 prediction:  [[2 3 3 3 3 3]] true 